# Import

In [68]:
import re
import pandas        as pd
import numpy         as np

# Load DataSet

In [69]:
df = pd.read_csv('../dataset/list_auto_cleaning.csv', encoding='utf-8', sep=';')

In [70]:
df.head()

,Unnamed: 0,id_ads,manufacturer_by,model,version,km,year,price,seller,endereco,scrapy_datetime
0,0,7fd72920-850d-43ba-a452-cbae1a6398da,land rover,defender,110 SW SE,81000,08-2015,38000,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03
1,1,840de0a3-82a9-454c-a725-508e211af7de,land rover,defender,Defender 90 td5 -MOTORE NUOVO-MECCANICA PERFETTA,147000,11-1999,24900,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03
2,2,6611c967-ab67-4db5-b28f-37b77fcd332f,land rover,defender,110 SW E,79000,08-2013,39900,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03
3,3,7484ee53-ebc5-4510-8bf9-032e657837cf,land rover,defender,110 SW E,78000,08-2012,39500,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03
4,4,08c56b66-b680-4f5c-b63a-33ce190b65eb,land rover,defender,110 SW SE,52000,08-2014,42600,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03


# Using REGEX langague to clean and capture data-info

In [71]:
df_clean = df.copy()

In [72]:
# Extraindo o codigo postal de cada país de origem do veiculo
# Aplicando Regex na limpeza
df_clean['zip_code'] = df_clean['endereco'].apply(lambda x: int(re.search('\d+', x).group(0) if pd.notnull( x ) else x ))

In [73]:
# Selecinando os modelos
df_clean['model'] = 'DEFENDER' + ' ' + df_clean['version'].apply(lambda x: ' '.join(re.findall(r'\b(88|90|110|130)\b', x)) if re.findall(r'\d+', x) else None)

In [74]:
# Version Type
df_clean['engine'] = df_clean['version'].apply(lambda x: ' '.join(re.findall(r'[A-Za-z]+\d+', x)) if re.findall(r'\d+', x) else None)
df_clean['engine_version'] = df_clean['version'].apply(lambda x: ' '.join(re.findall(r'(AWD|SW|SE|HSE|MHEV|HT)', x)) if re.findall(r'\d+', x) else None)
df_clean['cc_engine'] = df_clean['version'].apply(lambda x: ' '.join(re.findall(r'\b\d\.\d', x)) if re.findall(r'\d+', x) else None)

# Substituindo Valores
df_clean['engine'] = df_clean.apply(lambda row: row['engine'] + ' ' + row['engine_version'] + ' ' + row['cc_engine'] if row['engine'] is not None and row['engine_version'] is not None and row['cc_engine'] is not None else None, axis=1)


In [76]:
#Country
df_clean['country'] = df_clean['endereco'].apply(lambda x: str(re.search(r"([A-Z]{2})", x).group(0) if pd.notnull( x ) else x ))

In [77]:
df_clean.head(20)

,Unnamed: 0,id_ads,manufacturer_by,model,version,km,year,price,seller,endereco,scrapy_datetime,zip_code,engine,engine_version,cc_engine,country
0,0,7fd72920-850d-43ba-a452-cbae1a6398da,land rover,DEFENDER 110,110 SW SE,81000,08-2015,38000,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03,14440,SW SE,SW SE,,ES
1,1,840de0a3-82a9-454c-a725-508e211af7de,land rover,DEFENDER 90,Defender 90 td5 -MOTORE NUOVO-MECCANICA PERFETTA,147000,11-1999,24900,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03,14440,td5,,,ES
2,2,6611c967-ab67-4db5-b28f-37b77fcd332f,land rover,DEFENDER 110,110 SW E,79000,08-2013,39900,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03,14440,SW,SW,,ES
3,3,7484ee53-ebc5-4510-8bf9-032e657837cf,land rover,DEFENDER 110,110 SW E,78000,08-2012,39500,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03,14440,SW,SW,,ES
4,4,08c56b66-b680-4f5c-b63a-33ce190b65eb,land rover,DEFENDER 110,110 SW SE,52000,08-2014,42600,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03,14440,SW SE,SW SE,,ES
5,5,2604bbb4-f7ce-44d4-b03d-db78096ef722,land rover,DEFENDER 110,110 SW E,72000,08-2013,36800,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03,14440,SW,SW,,ES
6,6,cd9f42cb-9eea-46c5-b892-f6ca6a5757cd,land rover,NaN,Gancio traino + Verricello,285000,07-2007,15900,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03,14440,None,None,None,ES
7,7,140dd9d8-c26c-4bbd-804e-1ea25f78cd54,land rover,DEFENDER 90,90 Td4 | 1STE EIGENAAR | TOPSTAAT | LICHTE VRACHT,104527,04-2008,30995,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03,14440,Td4 HT HT,HT HT,,ES
8,8,6280ac39-eb76-440f-82cc-5f961d1e894d,land rover,DEFENDER 90,3.0 D200 MHEV S 90 3.0l 147 kW,17525,06-2022,66450,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03,14440,D200 MHEV 3.0 3.0,MHEV,3.0 3.0,ES
9,9,5725f30b-3f83-4124-8796-6528feb13ab0,land rover,DEFENDER 90,90 3.0D I6 250 CV AWD Auto X-Dynamic S tetto apr.,11734,10-2021,73800,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03,14440,I6 AWD 3.0,AWD,3.0,ES


In [ ]:
# pROXIMO PASSO → limpeza

# (x) Capturar info alfa-numericos da coluna 'version'
# (x) → Defender 90 (td5) -MOTORE NUOVO-MECCANICA PERFETTA
# (x) → 110 (Td4 Station Wagon SE) 7-Sitzer AHK
# (x) → (D250 X-Dynamic SE) 7 Pl. Camera 360 Roof rack
# (x) → (2.4 TD4 Station Wagon SE)

# (x) capturar info alfa-numerico da coluna 'endereco'
# (x) → ürgen Greil • (DE-94036) Passau
